In [54]:
!pip uninstall scikit-learn imbalanced-learn category_encoders -y
!pip install scikit-learn==1.3.2 imbalanced-learn==0.11.0 category_encoders==2.6.1


Found existing installation: scikit-learn 1.6.1
Uninstalling scikit-learn-1.6.1:
  Successfully uninstalled scikit-learn-1.6.1
Found existing installation: imbalanced-learn 0.13.0
Uninstalling imbalanced-learn-0.13.0:
  Successfully uninstalled imbalanced-learn-0.13.0
Found existing installation: category_encoders 2.8.1
Uninstalling category_encoders-2.8.1:
  Successfully uninstalled category_encoders-2.8.1
   ---------------------------------------- 0.0/9.1 MB ? eta -:--:--
   --- ------------------------------------ 0.8/9.1 MB 4.8 MB/s eta 0:00:02
   --------- ------------------------------ 2.1/9.1 MB 5.6 MB/s eta 0:00:02
   ------------- -------------------------- 3.1/9.1 MB 5.1 MB/s eta 0:00:02
   ------------------ --------------------- 4.2/9.1 MB 5.0 MB/s eta 0:00:01
   ------------------------ --------------- 5.5/9.1 MB 5.4 MB/s eta 0:00:01
   ------------------------------ --------- 7.1/9.1 MB 5.7 MB/s eta 0:00:01
   ---------------------------------------  8.9/9.1 MB 6.2 MB/s 

In [21]:
import sklearn
import imblearn
import category_encoders

print("scikit-learn:", sklearn.__version__)
print("imbalanced-learn:", imblearn.__version__)
print("category_encoders:", category_encoders.__version__)


scikit-learn: 1.6.1
imbalanced-learn: 0.13.0
category_encoders: 2.6.1


In [4]:
import pandas as pd
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from catboost import CatBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor 
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.svm import SVR
from sklearn.linear_model import Ridge, Lasso

In [5]:
df=pd.read_csv("tourism_sql.csv")

In [6]:
df['VisitMode'].unique()

array(['Friends', 'Family', 'Couples', 'Solo', 'Business'], dtype=object)

In [7]:
df.columns

Index(['UserId', 'ContinentId', 'Continent', 'RegionId', 'Region', 'CountryId',
       'Country', 'CityId', 'CityName', 'TransactionId', 'VisitYear',
       'VisitMonth', 'VisitModeId', 'VisitMode', 'AttractionId', 'Attraction',
       'AttractionCityId', 'AttractionAddress', 'AttractionTypeId',
       'AttractionType', 'Rating'],
      dtype='object')

In [8]:
columns_drop=['Continent','TransactionId','Region','Country','CityName','Attraction','AttractionAddress','AttractionType','VisitMode','VisitYear','ContinentId', 'RegionId']
df=df.drop(columns=columns_drop)

In [9]:
df['User_visit_count'] = df.groupby('UserId')['AttractionId'].transform('count')
df['Avg_user_rating'] = df.groupby('UserId')['Rating'].transform('mean')
df['Attraction_popularity'] = df.groupby('AttractionId')['Rating'].transform('count')

In [10]:
df.head()

,UserId,CountryId,CityId,VisitMonth,VisitModeId,AttractionId,AttractionCityId,AttractionTypeId,Rating,User_visit_count,Avg_user_rating,Attraction_popularity
0,23,22,154,9,4,748,1,72,5,1,5.000000,5806
1,34,22,167,7,3,640,1,63,4,1,4.000000,13192
2,60,14,52,7,2,748,1,72,5,1,5.000000,5806
3,103,22,106,4,2,640,1,63,4,3,4.666667,13192
4,103,22,106,4,2,841,1,92,5,3,4.666667,6426


In [11]:
df = df.drop('UserId', axis=1)

In [12]:
df = df.drop('VisitMonth', axis=1)

In [13]:
from sklearn.feature_selection import f_classif
import pandas as pd

# Make sure 'VisitMode' is label-encoded or already numeric
X = df.drop(['VisitModeId'], axis=1, errors='ignore')  # drop both if present
y = df['VisitModeId']  # use numeric label

# Compute ANOVA F-values for classification
f_values, p_values = f_classif(X, y)

# Create a summary DataFrame
feature_scores = pd.DataFrame({'Feature': X.columns, 'F-Score': f_values, 'P-Value': p_values})
print(feature_scores.sort_values(by='F-Score', ascending=False))


                 Feature     F-Score        P-Value
3       AttractionCityId  401.040051   0.000000e+00
2           AttractionId  288.147624  1.440690e-245
4       AttractionTypeId  172.964575  1.878088e-147
8  Attraction_popularity  130.509286  4.083149e-111
0              CountryId   72.383944   2.889967e-61
7        Avg_user_rating   39.594201   3.657097e-33
1                 CityId   36.178401   3.041069e-30
5                 Rating   35.277549   1.789027e-29
6       User_visit_count   25.025032   9.800989e-21


In [14]:

correlations = df.corr()['VisitModeId'].sort_values(ascending=False)
correlations = correlations.abs()
print(correlations)


VisitModeId              1.000000
AttractionCityId         0.125004
AttractionId             0.112152
User_visit_count         0.036595
AttractionTypeId         0.016873
Rating                   0.004981
Avg_user_rating          0.003183
CityId                   0.049042
Attraction_popularity    0.056866
CountryId                0.065583
Name: VisitModeId, dtype: float64


In [15]:
df.head()

,CountryId,CityId,VisitModeId,AttractionId,AttractionCityId,AttractionTypeId,Rating,User_visit_count,Avg_user_rating,Attraction_popularity
0,22,154,4,748,1,72,5,1,5.000000,5806
1,22,167,3,640,1,63,4,1,4.000000,13192
2,14,52,2,748,1,72,5,1,5.000000,5806
3,22,106,2,640,1,63,4,3,4.666667,13192
4,22,106,2,841,1,92,5,3,4.666667,6426


In [16]:
df['VisitModeId'].value_counts()

VisitModeId
2    21603
3    15208
4    10939
5     4517
1      623
Name: count, dtype: int64

In [41]:
df.columns


Index(['CountryId', 'CityId', 'VisitModeId', 'AttractionId',
       'AttractionCityId', 'AttractionTypeId', 'Rating', 'User_visit_count',
       'Avg_user_rating', 'Attraction_popularity'],
      dtype='object')

In [17]:
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split

X = df.drop("VisitModeId", axis=1)
y = df["VisitModeId"]

# Split first
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Apply SMOTE only to training data
smote = SMOTE(random_state=42)
X_res, y_res = smote.fit_resample(X_train, y_train)


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Train Random Forest on resampled data
rf = RandomForestClassifier(random_state=42)
rf.fit(X_res, y_res)

# 2. Predict on original test set (X_test)
y_pred = rf.predict(X_test)

# 3. Accuracy
print("✅ Accuracy:", accuracy_score(y_test, y_pred))

# 4. Classification report
print("\n📊 Classification Report:\n", classification_report(y_test, y_pred))

# 5. Confusion matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))




✅ Accuracy: 0.42584889964455874

📊 Classification Report:
               precision    recall  f1-score   support

           1       0.12      0.26      0.16       160
           2       0.52      0.50      0.51      5379
           3       0.47      0.45      0.46      3833
           4       0.31      0.31      0.31      2755
           5       0.24      0.28      0.26      1096

    accuracy                           0.43     13223
   macro avg       0.33      0.36      0.34     13223
weighted avg       0.43      0.43      0.43     13223

Confusion Matrix:
 [[  42   37   35   31   15]
 [  88 2686 1126 1010  469]
 [ 103 1112 1742  630  246]
 [  92  942  628  859  234]
 [  29  356  205  204  302]]
Accuracy: 0.42584889964455874
Precision: 0.434695893522732
Recall: 0.42584889964455874
F1 Score: 0.4296967572529146


In [38]:
from catboost import CatBoostClassifier

# Initialize and train (suppress verbose output)
cat = CatBoostClassifier(verbose=0, random_state=42)
cat.fit(X_res, y_res)

# Predict
y_pred_cat = cat.predict(X_test)

# Metrics
print("✅ CatBoost Accuracy:", accuracy_score(y_test, y_pred_cat))
print("📊 Classification Report (CatBoost):\n", classification_report(y_test, y_pred_cat))

# Confusion Matrix
cm_cat = confusion_matrix(y_test, y_pred_cat)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred_cat, average='weighted'))
print("Recall:", recall_score(y_test, y_pred_cat, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred_cat, average='weighted'))



✅ CatBoost Accuracy: 0.4524691824850639
📊 Classification Report (CatBoost):
               precision    recall  f1-score   support

           1       0.09      0.34      0.14       160
           2       0.51      0.67      0.58      5379
           3       0.50      0.42      0.46      3833
           4       0.36      0.20      0.26      2755
           5       0.22      0.17      0.19      1096

    accuracy                           0.45     13223
   macro avg       0.34      0.36      0.32     13223
weighted avg       0.45      0.45      0.44     13223

Confusion Matrix:
 [[  42   37   35   31   15]
 [  88 2686 1126 1010  469]
 [ 103 1112 1742  630  246]
 [  92  942  628  859  234]
 [  29  356  205  204  302]]
Precision: 0.4475827084390438
Recall: 0.4524691824850639
F1 Score: 0.4389290970047198


In [40]:
import pickle

# Save the model using pickle
with open("catboost_model_classification.pkl", "wb") as f:
    pickle.dump(cat, f)



In [23]:
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import precision_score,recall_score,f1_score,confusion_matrix,accuracy_score

In [19]:
#x=df.drop('VisitModeId',axis=1)
#y=df['VisitModeId']
#x_train,x_test,y_train,y_test =train_test_split(x,y,test_size=0.2, random_state=50)
#print(f"x_train {x_train.shape}")
#print(f"x_test {x_test.shape}")

In [20]:
models=[]
models.append(('naive_bayes', GaussianNB()))
models.append(('KNN', KNeighborsClassifier(n_neighbors=5)))
models.append(('Random_Forest', RandomForestClassifier()))
models.append(('Decision_tree', DecisionTreeClassifier()))
#models.append(('SVC', SVC(kernel='linear')))

In [21]:
for name,model in models:
    print(name,model)
    print()
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
    print("Accuracy:", accuracy_score(y_test, y_pred))
    print("Precision:", precision_score(y_test, y_pred, average='weighted'))
    print("Recall:", recall_score(y_test, y_pred, average='weighted'))
    print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
    print("----------------------------------------")

naive_bayes GaussianNB()

Confusion Matrix:
 [[  13   81   16   48    2]
 [  59 4389  515  340   76]
 [  72 2397 1001  320   43]
 [  89 1824  362  416   64]
 [  23  758   92  175   48]]
Accuracy: 0.44369658927626104
Precision: 0.4194691237665562
Recall: 0.44369658927626104
F1 Score: 0.39004812507568243
----------------------------------------
KNN KNeighborsClassifier()

Confusion Matrix:
 [[  18   76   41   21    4]
 [  23 3622 1126  496  112]
 [  16 1745 1629  382   61]
 [  33 1436  719  496   71]
 [  10  595  219  152  120]]
Accuracy: 0.445057853739696
Precision: 0.41960416655197263
Recall: 0.445057853739696
F1 Score: 0.4173818340957925
----------------------------------------
Random_Forest RandomForestClassifier()

Confusion Matrix:
 [[  28   49   47   28    8]
 [  15 3283 1092  757  232]
 [  18 1408 1824  471  112]
 [  25 1193  639  782  116]
 [   4  455  215  178  244]]
Accuracy: 0.46593057551236483
Precision: 0.45327595800231657
Recall: 0.46593057551236483
F1 Score: 0.45585401955

In [40]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

param_dist = {
    'n_estimators': randint(100, 300),
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2],
    'class_weight': ['balanced', 'balanced_subsample']
}

rf = RandomForestClassifier(random_state=42)
random_search = RandomizedSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    n_iter=20,  # Try 20 random combos
    cv=3,
    scoring='f1_weighted',
    random_state=42,
    n_jobs=-1,
    verbose=2
)

random_search.fit(X_train, y_train)


Fitting 3 folds for each of 20 candidates, totalling 60 fits


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(random_state=42),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'class_weight': ['balanced',
                                                         'balanced_subsample'],
                                        'max_depth': [10, 20, None],
                                        'min_samples_leaf': [1, 2],
                                        'min_samples_split': [2, 5],
                                        'n_estimators': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x0000015D3518D070>},
                   random_state=42, scoring='f1_weighted', verbose=2)

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Get the best model
best_rf = random_search.best_estimator_

# Predict on test data
y_pred = best_rf.predict(X_test)

# Print metrics
print("Best Parameters:\n", random_search.best_params_)
print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred, average='weighted'))
print("Recall:", recall_score(y_test, y_pred, average='weighted'))
print("F1 Score:", f1_score(y_test, y_pred, average='weighted'))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


Best Parameters:
 {'class_weight': 'balanced', 'max_depth': 20, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 187}

Accuracy: 0.4378734024049005

Classification Report:
               precision    recall  f1-score   support

           1       0.11      0.30      0.16       160
           2       0.54      0.52      0.53      5379
           3       0.50      0.46      0.48      3833
           4       0.33      0.30      0.32      2755
           5       0.23      0.33      0.27      1096

    accuracy                           0.44     13223
   macro avg       0.34      0.38      0.35     13223
weighted avg       0.45      0.44      0.44     13223


Confusion Matrix:
 [[  48   34   38   25   15]
 [ 115 2794  997  870  603]
 [ 132 1085 1755  574  287]
 [ 111  947  577  834  286]
 [  35  356  154  192  359]]
